In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
GOOG_path = Path("../files/GOOG.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
goog_df = pd.read_csv(GOOG_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([goog_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,26.307772,26.327717,26.054466,26.260900,26.260900,35023895.0,13.41,188.309998,0.09,25.428186
2014-05-01,26.283339,26.573542,26.122280,26.494759,26.494759,38110345.0,13.25,188.330002,0.09,25.480045
2014-05-02,26.614929,26.626896,26.208544,26.324226,26.324226,33770463.0,12.91,188.059998,0.09,25.865486
2014-05-05,26.169153,26.372595,25.994633,26.318243,26.318243,20482080.0,13.29,188.419998,0.09,26.423454
2014-05-06,26.189596,26.268379,25.682489,25.686478,25.686478,33780490.0,13.80,186.779999,0.09,26.581520
2014-05-07,25.718889,25.763268,25.096100,25.428186,25.428186,64486563.0,13.40,187.880005,0.09,26.260403
2014-05-08,25.353392,25.790691,25.253166,25.480045,25.480045,40426688.0,13.43,187.679993,0.09,25.927814
2014-05-09,25.467579,25.923826,25.140976,25.865486,25.865486,48789585.0,12.92,187.960007,0.09,25.960226
2014-05-12,26.103832,26.436916,25.879448,26.423454,26.423454,38250730.0,12.23,189.789993,0.09,26.370600
2014-05-13,26.471823,26.730112,26.403011,26.581520,26.581520,33068541.0,12.13,189.960007,0.09,26.415976


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,149.369995,152.929993,148.139999,148.479996,148.479996,47676700.0,14.33,512.859985,5.33,151.149994,139.858999,62.102838,148.428297,139.858999,131.289701
2024-03-19,148.979996,149.619995,147.009995,147.919998,147.919998,17748400.0,13.82,515.710022,5.33,151.699997,140.144999,60.949348,149.397818,140.144999,130.892180
2024-03-20,148.789993,149.759995,147.664993,149.679993,149.679993,17730000.0,13.04,520.479980,5.33,151.940002,140.436999,63.259083,150.512790,140.436999,130.361207
2024-03-21,150.320007,151.304993,148.009995,148.740005,148.740005,19843900.0,12.92,522.200012,5.33,152.259995,140.607999,61.177836,151.138544,140.607999,130.077453
2024-03-22,150.240005,152.559998,150.089996,151.770004,151.770004,19226300.0,13.06,521.210022,5.33,156.500000,140.931999,65.157240,152.423951,140.931999,129.440047


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,35023895.0,13.41,188.309998,0.09,25.428186,NaN,NaN,NaN,NaN,NaN
2014-05-01,38110345.0,13.25,188.330002,0.09,25.480045,NaN,100.000000,NaN,NaN,NaN
2014-05-02,33770463.0,12.91,188.059998,0.09,25.865486,NaN,56.012800,NaN,NaN,NaN
2014-05-05,20482080.0,13.29,188.419998,0.09,26.423454,NaN,55.097106,NaN,NaN,NaN
2014-05-06,33780490.0,13.80,186.779999,0.09,26.581520,NaN,19.271319,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/GOOG_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,33040464.0,11.68,191.380005,0.09,27.158438,26.481819,72.498592,28.005806,26.481819,24.957833
2014-05-29,27082150.0,11.57,192.369995,0.09,27.619171,26.565141,70.666054,28.215268,26.565141,24.915013
2014-05-30,35422988.0,11.40,192.679993,0.09,27.740339,26.636295,70.438360,28.392936,26.636295,24.879654
2014-06-02,28700582.0,11.58,192.899994,0.10,28.029045,26.701117,63.523936,28.504334,26.701117,24.897900
2014-06-03,37332215.0,11.87,192.800003,0.10,27.950762,26.743825,54.787666,28.549319,26.743825,24.938331


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
Average R-squared (Train): -3.2869022722029415
Average R-squared (Test): -3.300284249892711


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step
Model 1 - Train R-squared: 0.9451697577976257, Test R-squared: 0.95306395706548, Train MAE: 6.608263363329633, Train MSE: 61.28083861628059, Test MAE: 6.5661568069210245, Test MSE: 61.43141861618415
Model 2 - Train R-squared: 0.903762135241002, Test R-squared: 0.9048228069272932, Train MAE: 8.43319260122563, Train MSE: 106.54196012659926, Test MAE: 9.475982178744777, Test MSE: 131.72181447299562
Model 3 - Train R-squared: 0.9230585109910914, Test R-squared: 0.9259225317629651, Train MAE: 7.975115000928717, Train MSE: 87.25518534197828, Test MAE: 7.907264943214946, Test MSE: 86.09587712358775
Model 4 - Train R-squared: 0.9339472447379169, Test R-squared: 0.9378098287088241, Train MAE: 7.093743205577375, Train MSE: 74.17508959770002, Test MAE: 7.147177974408743, Test MSE: 78.40728490493703
Model 5 - Train R-squared: 0.9074304253670051, Test R-squared: 0.8980540503266411, Train MAE: 8.624722867289181, Train MS

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.903762135241002, Test R-squared: 0.9048228069272932, Train MAE: 8.43319260122563, Train MSE: 106.54196012659926, Test MAE: 9.475982178744777, Test MSE: 131.72181447299562


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step


,Actual,Predicted
date,,
2014-05-28,28.007107,23.546053
2014-05-29,27.927326,23.668116
2014-05-30,27.917852,23.653910
2014-06-02,27.620667,23.682898
2014-06-03,27.172398,23.498177
...,...,...
2024-03-18,148.479996,104.215431
2024-03-19,147.919998,106.273567
2024-03-20,149.679993,106.693489


In [22]:
predictions_df.to_csv('../predicted_data/GOOG_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
                Actual   Predicted
date                              
2014-05-28   28.007107   23.546053
2014-05-29   27.927326   23.668116
2014-05-30   27.917852   23.653910
2014-06-02   27.620667   23.682898
2014-06-03   27.172398   23.498177
...                ...         ...
2024-03-18  148.479996  104.215431
2024-03-19  147.919998  106.273567
2024-03-20  149.679993  106.693489
2024-03-21  148.740005  107.018959
2024-03-22  151.770004  107.813835

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,28.007107,23.546053,-15.928293
2014-05-29,27.927326,23.668116,-15.251050
2014-05-30,27.917852,23.653910,-15.273175
2014-06-02,27.620667,23.682898,-14.256605
2014-06-03,27.172398,23.498177,-13.521889
...,...,...,...
2024-03-18,148.479996,104.215431,-29.811804
2024-03-19,147.919998,106.273567,-28.154699
2024-03-20,149.679993,106.693489,-28.718938


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 29.876192746926638
